In [3]:
import sys, os
sys.path.insert(1, os.getcwd() + '\..\src')
sys.path.insert(1, os.getcwd() + '/../src')

In [4]:
from session_generator import *
from pev_digital_twin import *

# Main Simulation

In [5]:
# List with number of sessions for different days ([number of sessions on day 1, ..., number of sessions on the last day])
daily_sessions = [10]*10

try:
    input_gen = InputGen(daily_sessions=daily_sessions, data_file='..\data\Sessions2_20221020.csv', rnd_seeds=(4,5,30))
except:
    input_gen = InputGen(daily_sessions=daily_sessions, data_file='../data/Sessions2_20221020.csv', rnd_seeds=(4,5,30))# Remove outliers:
input_gen.data = input_gen.data[(input_gen.data['DurationHrs'] < 15) & (input_gen.data['DurationHrs'] > 1/6)]
input_gen.data = input_gen.data[input_gen.data['cumEnergy_Wh'] / input_gen.data['DurationHrs'] <= 6700]

input_gen.arrival_gen()
input_gen.duration_gen()
input_gen.energy_gen()

input_df = input_gen.df

#input_df['arrivalMin'] = input_df['arrivalMin'].apply(lambda x: round(x/30)*30 + 30)
#input_df['arrivalMinGlobal'] = input_df['arrivalMinGlobal'].apply(lambda x: round(x/30)*30 + 30)
input_df['durationMin'] = input_df['durationMin'].apply(lambda x: round(x/30)*30 + 90)
input_df['cumEnergy_Wh'] = input_df['cumEnergy_Wh'].apply(lambda x: round(x/10)*10 + 10)

input_df.head(10)

,arrivalDay,arrivalMin,arrivalMinGlobal,durationMin,cumEnergy_Wh
0,0,98,98,180,20
1,0,475,475,600,40
2,0,487,487,270,30
3,0,544,544,630,20
4,0,579,579,390,40
5,0,647,647,450,30
6,0,659,659,360,40
7,0,1115,1115,270,20
8,0,1148,1148,210,20
9,0,1165,1165,150,10


In [6]:
import simpy

In [7]:
# SIMULATION PARAMETERS
NUM_DAYS = len(daily_sessions)
SIM_RUN_TIME = 1440*NUM_DAYS
CAR_ARR_TIME = 120
CAR_STAY_TIME = 300

env = simpy.Environment()

env.process(first_process(env, input_df, SIM_RUN_TIME))   
env.run(SIM_RUN_TIME + 10)

curr_time =  98
departure_time =  278
requested_energy =  20
curr_time =  475
departure_time =  1075
requested_energy =  40
curr_time =  487
departure_time =  757
requested_energy =  30
curr_time =  544
departure_time =  1174
requested_energy =  20
curr_time =  579
departure_time =  969
requested_energy =  40
curr_time =  647
departure_time =  1097
requested_energy =  30
curr_time =  659
departure_time =  1019
requested_energy =  40
curr_time =  1115
departure_time =  1385
requested_energy =  20


TypeError: 'NoneType' object is not subscriptable

# Output Table

Generate dataframe

In [ ]:
df = pd.DataFrame({'session': session, 'choice': user_choice, 'arrival time': arrival_time, 'departure time': departure_time})
df['durationHrs'] = (df['departure time'] - df['arrival time']) / 60
df['arrival time'] = pd.Timestamp(2022, 4, 4, 9) + pd.TimedeltaIndex(df['arrival time'], unit='m')
df['departure time'] = pd.Timestamp(2022, 4, 4, 9) + pd.TimedeltaIndex(df['departure time'], unit='m')

In [ ]:
df['vehicle_model'] = car_type

df['energy_requested'] = energyreq

df['rate_scheduled'] = rate_flex
df['rate_regular'] = rate_asap

df['sch_centsPerHr'] = ((df['rate_scheduled']*100)/df['durationHrs']).round(decimals=0)
df['reg_centsPerHr'] = ((df['rate_regular']*100)/df['durationHrs']).round(decimals=0)

df['energyReq_Wh'] = df['energy_requested']*1000

df = df.drop('rate_scheduled', 1)
df = df.drop('rate_regular', 1)
df = df.drop('energy_requested', 1)

df.head()

In [ ]:
def hr_func(ts):
    return ts.hour
df['arrival_hour'] = df['arrival time'].apply(hr_func)
df['departure_hour'] = df['departure time'].apply(hr_func)

In [ ]:
df_hist = pd.DataFrame(df['arrival_hour'])
df_hist['departure_hour']=df['departure_hour']

# plot melted dataframe in a single command
p=sns.histplot(df_hist.melt(), x='value', hue='variable',
             multiple='dodge', shrink=.75, bins=20)

p.set_xlabel("hour", fontsize = 10)
p.set_ylabel("frequency", fontsize = 10);

In [ ]:
df['vehicle_model'].value_counts().plot(kind='barh', figsize=(10, 10));

In [ ]:
df = df[df.choice != 'OCCUPIED']

View consolidated simulation dataframe

In [ ]:
df.head()

Arrival and departure frequency plot

In [ ]:
df_hist = pd.DataFrame(df['arrival_hour'])
df_hist['departure_hour']=df['departure_hour']

# plot melted dataframe in a single command
p=sns.histplot(df_hist.melt(), x='value', hue='variable',
             multiple='dodge', shrink=.75, bins=20)

p.set_xlabel("hour", fontsize = 10)
p.set_ylabel("frequency", fontsize = 10);

Plot of vehicle types

In [ ]:
df['vehicle_model'].value_counts().plot(kind='barh', figsize=(10, 10));

In [ ]:
df['choice'].value_counts()

Distribution of user choice

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'REGULAR', 'SCHEDULED', 'LEAVE'
sizes = [64, 22, 8]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

Plot session times

In [ ]:
sns.boxplot(x= df['durationHrs']) 
plt.title("Distribution of session times");

Plot energy Requirements

In [ ]:
sns.boxplot(x= df['energyReq_Wh']) 
plt.title("Distribution of Energy Required");